In [ ]:
from python.read_pics import *
import os

In [ ]:
files = []
for name in os.listdir("data"):
    if name.endswith(".bin"):
        files.append(name)

In [ ]:
import sys
import numpy as np
from matplotlib import pyplot

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from sklearn.model_selection import train_test_split

In [ ]:
labels = ['NOKEY', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B' ,'C', 'D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','CTRL','ENTER','SHIFT','SPACE','SUPPR']

len_labels = len(labels)

input_set = []
result_set = []
index = 0

for label in labels:
    pics, info = get_pics_from_file('data/pics_' + label + '.bin')
    input_set = input_set + pics[0:6963]
    tmp = [0] * len_labels
    tmp[index] = 1
    index += 1
    #for i in range(info["nb_trames"] - 1):
    for i in range(6963):
        result_set.append(tmp)

input_set = np.array(input_set)
result_set = np.array(result_set)

In [ ]:
def predict(file, frame):
    model = tf.keras.models.load_model(file)

    model.compile(
        optimizer=opt,
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    
    array = tf.expand_dims(frame, 0)

    predictions = model.predict_on_batch(array)
    return predictions

In [ ]:
def __predict(model, frame):
    array = tf.expand_dims(frame, 0)

    predictions = model.predict_on_batch(array)
    return predictions

In [ ]:
es_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=3)

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
opt = SGD(lr=0.1, momentum=0.9)
input_train, input_test, output_train, output_test = train_test_split(input_set, result_set, test_size=0.33)

# determine the number of input features
n_features = X_train.shape[1]
# define model
model = Sequential()
model.add(Dense(17, activation='relu', input_shape=(n_features,)))
#model.add(Dropout(0.1))
model.add(Dense(3000, activation='relu'))
#model.add(Dropout(0.1))
model.add(Dense(42, activation='sigmoid'))
# compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=["accuracy"])
# fit the model
model.fit(input_train, output_train, epochs=1000, batch_size=32, verbose=1, validation_data=(input_test, output_test), callbacks=callbacks)
# evaluate the model
error = model.evaluate(input_test, output_test)
model.save("weight.h5")


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]

input_train, input_test, output_train, output_test = train_test_split(input_set, result_set, test_size=0.33)


model = tf.keras.models.load_model("E:/Hackaton/audioKeyLoger/53percent.h5")
# compile the model
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=["accuracy"])
# fit the model
model.fit(input_train, output_train, epochs=1000, batch_size=32, verbose=1, validation_data=(input_test, output_test), callbacks=callbacks)
# evaluate the model
error = model.evaluate(input_test, output_test)
model.save("weight.h5")

In [ ]:
def debrief(predictions):
    index = np.argmax(predictions)
    return labels[index], index

In [ ]:
def keylogger():
    
    model = tf.keras.models.load_model("58percent.h5")
    model.compile(
        optimizer=opt,
        loss="binary_crossentropy",
        metrics=["accuracy"])
    
    frames, info = get_pics_from_file('pics_LOGINMDP.bin')
    histo = [0 for j in range(42)]
    result = []

    for i in range(info["nb_trames"] - 1):
        __predict(model, frames[i])
        _, index = debrief(__predict(model, frames[i]))
        histo[index] += 1
        if i % 15 == 0:
            print(i / info["nb_trames"])
            #if result == [] or result[-1] != labels[np.argmax(histo)]:
            m = np.argmax(histo)
            result.append(labels[m])
            histo[m] = 0
            #m = np.argmax(histo)
            #result[-1].append(labels[m])
            histo = [0 for j in range(42)]
    return result

In [ ]:
r = keylogger()

In [ ]:
r